<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
import tensorflow as tf
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [2]:
# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
imgtrain_dir = "/content/train/images/"
imgtrainmask_dir = "/content/train/masks/"
csvtrain_file = "/content/train.csv"

imgtest_dir = "/content/train.csv"
csvtest_file = "/content/test.csv"
# Leer el fichero CSV con las etiquetas
df = pd.read_csv(csvtrain_file, dtype={"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype={"class": "category"})

# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
df['class_encoded'] = label_encoder.fit_transform(df['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(df['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot y la columna 'mask_filename'
df = pd.concat([df[['image_filename', 'mask_filename']], class_one_hot], axis=1)
df


,image_filename,mask_filename,normal,benign,malignant
0,image_003.png,mask_003.png,True,False,False
1,image_004.png,mask_004.png,True,False,False
2,image_008.png,mask_008.png,True,False,False
3,image_012.png,mask_012.png,True,False,False
4,image_016.png,mask_016.png,True,False,False
...,...,...,...,...,...
332,image_399.png,mask_399.png,False,False,True
333,image_400.png,mask_400.png,False,False,True
334,image_416.png,mask_416.png,False,False,True
335,image_419.png,mask_419.png,False,False,True


In [3]:
# Dividir el conjunto en entrenamiento, validación y test (70:15:15)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [4]:
archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [6]:
# Dimensiones deseadas de la imagen
img_width, img_height = 244,244
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

def load_and_preprocess_image(image_filename, label):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    image = tf.image.resize(image, [img_width, img_height])
    image = image / 255.0
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_paths = df[x_col].values
    labels = df[y_col].values
    dataset = Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')



Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [ ]:
import matplotlib.pyplot as plt

# Función para mostrar las imágenes del batch con sus etiquetas
def show_images(dataset):
    # Itera sobre el conjunto de datos
    for images, labels in dataset:
        # Itera sobre cada imagen y su etiqueta en el lote
        for image, label in zip(images, labels):
            # Muestra la imagen
            plt.imshow(image)
            plt.title('Etiqueta: {}'.format(label.numpy()))  # Convierte la etiqueta a numpy array para imprimir
            plt.axis('off')  # Desactiva los ejes
            plt.show()

# Muestra las imágenes del conjunto de entrenamiento
show_images(train_dataset)


In [18]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

def get_model(input_shape, num_classes):
    # Cargar la base convolucional del modelo VGG16 pre-entrenado en ImageNet
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Congelar los pesos de las capas convolucionales del modelo base
    for layer in base_model.layers:
        layer.trainable = False

    # Crear un modelo secuencial y agregar las capas del modelo base
    model = Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    return model

# Definir el tamaño de entrada y el número de clases
input_shape = (img_width, img_height, n_channels)
num_classes = 3  # Normal, Benigno, Maligno
model = get_model(input_shape, num_classes)

# Definir la métrica F1
def f1_metric(y_true, y_pred):
    y_true_int = tf.argmax(y_true, axis=1)
    y_pred_int = tf.argmax(y_pred, axis=1)
    y_true_int = tf.cast(y_true_int, tf.float32)
    y_pred_int = tf.cast(y_pred_int, tf.float32)

    true_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 1), tf.equal(y_pred_int, 1)), tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 0), tf.equal(y_pred_int, 1)), tf.float32))
    false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 1), tf.equal(y_pred_int, 0)), tf.float32))

    precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
    recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())
    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

    return f1

           # optimizador Adam
model.compile(loss='categorical_crossentropy',  # función de pérdida para problemas de clasificación multi-clase
              optimizer='adam',
              metrics=['accuracy'])

# Imprimir el resumen del modelo
model.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 512)               12845568  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                       

In [19]:
# Definir el número de épocas
epochs = 10

# Entrenar el modelo
history = model.fit(train_dataset,
                    epochs=epochs,
                    verbose = 2,
                    validation_data=val_dataset)

Epoch 1/10
8/8 - 219s - loss: 3.2058 - accuracy: 0.4093 - val_loss: 2.4271 - val_accuracy: 0.1600 - 219s/epoch - 27s/step
Epoch 2/10
8/8 - 211s - loss: 3.1060 - accuracy: 0.5148 - val_loss: 1.4221 - val_accuracy: 0.3800 - 211s/epoch - 26s/step
Epoch 3/10
8/8 - 210s - loss: 2.1938 - accuracy: 0.4768 - val_loss: 0.7062 - val_accuracy: 0.7600 - 210s/epoch - 26s/step
Epoch 4/10
8/8 - 204s - loss: 1.3655 - accuracy: 0.5654 - val_loss: 0.7043 - val_accuracy: 0.7400 - 204s/epoch - 26s/step
Epoch 5/10
8/8 - 211s - loss: 0.9333 - accuracy: 0.5949 - val_loss: 0.7376 - val_accuracy: 0.6800 - 211s/epoch - 26s/step
Epoch 6/10
8/8 - 210s - loss: 0.6848 - accuracy: 0.6920 - val_loss: 0.5951 - val_accuracy: 0.8000 - 210s/epoch - 26s/step
Epoch 7/10
8/8 - 210s - loss: 0.6541 - accuracy: 0.7089 - val_loss: 0.6739 - val_accuracy: 0.7600 - 210s/epoch - 26s/step
Epoch 8/10
8/8 - 211s - loss: 0.6124 - accuracy: 0.7679 - val_loss: 0.5924 - val_accuracy: 0.7800 - 211s/epoch - 26s/step
Epoch 9/10
8/8 - 209s - 

In [20]:
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=2)
print("test_loss: %.4f, test_acc: %.4f" % (test_loss, test_accuracy))

2/2 - 36s - loss: 0.6555 - accuracy: 0.7600 - 36s/epoch - 18s/step
test_loss: 0.6555, test_acc: 0.7600


In [21]:
# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 1s 635ms/step
   image_filename  normal  benign  malignant
5   image_005.png       1       0          0
44  image_006.png       1       0          0
7   image_007.png       1       0          0
70  image_009.png       1       0          0
43  image_010.png       1       0          0
